# 📊 TELECOM X - CHURN ANALYSIS PROJECT
## 🔄 Notebook 01: ETL Pipeline (Extract, Transform, Load)

---

### 🎯 **Objetivos:**
1. 📥 **Extraer** datos de múltiples fuentes (APIs, CSV, mock data)
2. 🔧 **Transformar** y limpiar los datos
3. ✅ **Validar** calidad de datos
4. 💾 **Cargar** datos procesados
5. 🌐 **Enriquecer** con APIs externas

---

**👤 Desarrollado por:** Elizabeth Díaz Familia  
**📅 Fecha:** 2025

---

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("📦 Librerías importadas exitosamente")
print(f"🐍 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")

## 📥 1. EXTRACCIÓN DE DATOS

In [ ]:
print("=" * 70)
print("📥 FASE 1: EXTRACCIÓN DE DATOS")
print("=" * 70)

# Opción 1: Generar datos mock para demostración
def generate_mock_telecom_data(n_customers=1000):
    """Generar datos mock de clientes de telecomunicaciones"""
    
    np.random.seed(42)
    
    data = {
        'CustomerID': [f'CUST{i:05d}' for i in range(1, n_customers + 1)],
        'Gender': np.random.choice(['Male', 'Female'], n_customers),
        'SeniorCitizen': np.random.choice([0, 1], n_customers, p=[0.84, 0.16]),
        'Partner': np.random.choice(['Yes', 'No'], n_customers, p=[0.48, 0.52]),
        'Dependents': np.random.choice(['Yes', 'No'], n_customers, p=[0.30, 0.70]),
        'tenure': np.random.randint(1, 73, n_customers),
        'PhoneService': np.random.choice(['Yes', 'No'], n_customers, p=[0.90, 0.10]),
        'MultipleLines': np.random.choice(['Yes', 'No', 'No phone service'], n_customers),
        'InternetService': np.random.choice(['DSL', 'Fiber optic', 'No'], n_customers, p=[0.34, 0.44, 0.22]),
        'OnlineSecurity': np.random.choice(['Yes', 'No', 'No internet'], n_customers),
        'OnlineBackup': np.random.choice(['Yes', 'No', 'No internet'], n_customers),
        'DeviceProtection': np.random.choice(['Yes', 'No', 'No internet'], n_customers),
        'TechSupport': np.random.choice(['Yes', 'No', 'No internet'], n_customers),
        'StreamingTV': np.random.choice(['Yes', 'No', 'No internet'], n_customers),
        'StreamingMovies': np.random.choice(['Yes', 'No', 'No internet'], n_customers),
        'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], n_customers, p=[0.55, 0.21, 0.24]),
        'PaperlessBilling': np.random.choice(['Yes', 'No'], n_customers, p=[0.59, 0.41]),
        'PaymentMethod': np.random.choice(
            ['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card'],
            n_customers,
            p=[0.34, 0.23, 0.22, 0.21]
        ),
        'MonthlyCharges': np.random.uniform(18.0, 120.0, n_customers),
    }
    
    df = pd.DataFrame(data)
    
    # TotalCharges basado en tenure y MonthlyCharges
    df['TotalCharges'] = df['tenure'] * df['MonthlyCharges'] + np.random.uniform(-50, 50, n_customers)
    df['TotalCharges'] = df['TotalCharges'].clip(lower=0)
    
    # Churn con lógica realista
    churn_prob = 0.27  # base probability
    
    # Factores que afectan churn
    churn_factors = np.zeros(n_customers)
    churn_factors += (df['Contract'] == 'Month-to-month').astype(int) * 0.3
    churn_factors += (df['tenure'] < 12).astype(int) * 0.2
    churn_factors += (df['InternetService'] == 'Fiber optic').astype(int) * 0.1
    churn_factors += (df['SeniorCitizen'] == 1).astype(int) * 0.1
    
    df['Churn'] = np.random.random(n_customers) < (churn_prob + churn_factors * 0.5)
    df['Churn'] = df['Churn'].map({True: 'Yes', False: 'No'})
    
    return df

# Generar datos
df_raw = generate_mock_telecom_data(n_customers=1000)

print(f"\n✅ Datos generados: {len(df_raw):,} registros")
print(f"📊 Columnas: {len(df_raw.columns)}")
print(f"\n📋 Primeras filas:")
display(df_raw.head())

## 🔍 2. ANÁLISIS INICIAL DE CALIDAD

In [ ]:
print("=" * 70)
print("🔍 ANÁLISIS INICIAL DE CALIDAD")
print("=" * 70)

# Información general
print("\n📊 Información del Dataset:")
print(df_raw.info())

# Valores nulos
print("\n🔎 Valores Nulos:")
null_counts = df_raw.isnull().sum()
if null_counts.sum() > 0:
    print(null_counts[null_counts > 0])
else:
    print("✅ No hay valores nulos")

# Duplicados
duplicates = df_raw.duplicated().sum()
print(f"\n📋 Registros Duplicados: {duplicates}")

# Estadísticas descriptivas
print("\n📈 Estadísticas Descriptivas:")
display(df_raw.describe())

## 🔧 3. TRANSFORMACIÓN DE DATOS

In [ ]:
print("=" * 70)
print("🔧 FASE 2: TRANSFORMACIÓN DE DATOS")
print("=" * 70)

# Copiar dataframe
df_transformed = df_raw.copy()

# 1. Estandarizar nombres de columnas
print("\n1️⃣ Estandarizando nombres de columnas...")
df_transformed.columns = df_transformed.columns.str.strip()

# 2. Convertir tipos de datos
print("2️⃣ Convirtiendo tipos de datos...")
df_transformed['SeniorCitizen'] = df_transformed['SeniorCitizen'].astype(str).map({'0': 'No', '1': 'Yes'})
df_transformed['MonthlyCharges'] = df_transformed['MonthlyCharges'].round(2)
df_transformed['TotalCharges'] = df_transformed['TotalCharges'].round(2)

# 3. Crear variables derivadas
print("3️⃣ Creando variables derivadas...")

# Tenure groups
df_transformed['TenureGroup'] = pd.cut(
    df_transformed['tenure'],
    bins=[0, 12, 24, 48, 73],
    labels=['0-12 months', '12-24 months', '24-48 months', '48+ months']
)

# Charges groups
df_transformed['ChargesGroup'] = pd.cut(
    df_transformed['MonthlyCharges'],
    bins=[0, 35, 70, 120],
    labels=['Low', 'Medium', 'High']
)

# Total services
service_cols = ['PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
df_transformed['TotalServices'] = 0
for col in service_cols:
    if col in df_transformed.columns:
        df_transformed['TotalServices'] += (df_transformed[col] == 'Yes').astype(int)

# Customer Lifetime Value (CLV) estimate
df_transformed['CLV_Estimate'] = df_transformed['TotalCharges'] * 1.2  # Simple estimate

print("\n✅ Transformaciones completadas")
print(f"📊 Nuevas columnas: {len(df_transformed.columns) - len(df_raw.columns)}")

## 🌐 4. ENRIQUECIMIENTO CON APIs EXTERNAS

In [ ]:
print("=" * 70)
print("🌐 FASE 3: ENRIQUECIMIENTO CON APIS")
print("=" * 70)

# API 1: Exchange Rates
print("\n💱 Obteniendo tasas de cambio...")
try:
    response = requests.get('https://api.exchangerate-api.com/v4/latest/USD', timeout=10)
    if response.status_code == 200:
        exchange_data = response.json()
        usd_to_eur = exchange_data['rates'].get('EUR', 0.85)
        
        # Agregar columnas con conversión
        df_transformed['MonthlyCharges_EUR'] = (df_transformed['MonthlyCharges'] * usd_to_eur).round(2)
        df_transformed['TotalCharges_EUR'] = (df_transformed['TotalCharges'] * usd_to_eur).round(2)
        
        print(f"✅ Tasa USD/EUR aplicada: {usd_to_eur}")
    else:
        print("⚠️ No se pudo obtener tasas de cambio")
except Exception as e:
    print(f"⚠️ Error en API de tasas: {str(e)}")

# Agregar timestamp de procesamiento
df_transformed['DataProcessedAt'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

print("\n✅ Enriquecimiento completado")

## ✅ 5. VALIDACIÓN DE DATOS

In [ ]:
print("=" * 70)
print("✅ FASE 4: VALIDACIÓN DE DATOS")
print("=" * 70)

validation_results = {
    'total_records': len(df_transformed),
    'total_columns': len(df_transformed.columns),
    'null_values': df_transformed.isnull().sum().sum(),
    'duplicates': df_transformed.duplicated().sum(),
    'churn_distribution': df_transformed['Churn'].value_counts().to_dict()
}

print("\n📊 Resultados de Validación:")
print(f"✓ Total de registros: {validation_results['total_records']:,}")
print(f"✓ Total de columnas: {validation_results['total_columns']}")
print(f"✓ Valores nulos: {validation_results['null_values']}")
print(f"✓ Duplicados: {validation_results['duplicates']}")
print(f"\n✓ Distribución de Churn:")
for status, count in validation_results['churn_distribution'].items():
    percentage = (count / validation_results['total_records']) * 100
    print(f"   {status}: {count:,} ({percentage:.1f}%)")

# Validaciones específicas
print("\n🔍 Validaciones Específicas:")

# 1. MonthlyCharges debe ser > 0
invalid_charges = (df_transformed['MonthlyCharges'] <= 0).sum()
print(f"✓ MonthlyCharges inválidos: {invalid_charges}")

# 2. Tenure debe estar en rango válido
invalid_tenure = ((df_transformed['tenure'] < 0) | (df_transformed['tenure'] > 72)).sum()
print(f"✓ Tenure inválidos: {invalid_tenure}")

# 3. CustomerID único
unique_customers = df_transformed['CustomerID'].nunique()
print(f"✓ CustomerID únicos: {unique_customers}/{len(df_transformed)}")

if validation_results['null_values'] == 0 and validation_results['duplicates'] == 0:
    print("\n🎉 ¡Validación exitosa! Datos listos para análisis.")
else:
    print("\n⚠️ Se encontraron problemas en la validación.")

## 💾 6. CARGA DE DATOS PROCESADOS

In [ ]:
print("=" * 70)
print("💾 FASE 5: CARGA DE DATOS")
print("=" * 70)

# Crear directorios si no existen
Path('data/raw').mkdir(parents=True, exist_ok=True)
Path('data/processed').mkdir(parents=True, exist_ok=True)

# Guardar datos raw
raw_path = 'data/raw/telecom_churn_raw.csv'
df_raw.to_csv(raw_path, index=False)
print(f"\n✓ Datos raw guardados: {raw_path}")
print(f"  Tamaño: {len(df_raw):,} registros")

# Guardar datos procesados
processed_path = 'data/processed/telecom_churn_clean.csv'
df_transformed.to_csv(processed_path, index=False)
print(f"\n✓ Datos procesados guardados: {processed_path}")
print(f"  Tamaño: {len(df_transformed):,} registros")
print(f"  Columnas: {len(df_transformed.columns)}")

# Guardar también en formato parquet (más eficiente)
parquet_path = 'data/processed/telecom_churn_clean.parquet'
df_transformed.to_parquet(parquet_path, index=False)
print(f"\n✓ Datos guardados en Parquet: {parquet_path}")

# Guardar metadata
metadata = {
    'processed_at': datetime.now().isoformat(),
    'total_records': len(df_transformed),
    'total_columns': len(df_transformed.columns),
    'columns': list(df_transformed.columns),
    'churn_rate': (df_transformed['Churn'] == 'Yes').mean(),
    'validation': validation_results
}

metadata_path = 'data/processed/metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=4)
print(f"\n✓ Metadata guardada: {metadata_path}")

print("\n🎉 ¡Pipeline ETL completado exitosamente!")

## 📊 7. RESUMEN DEL PIPELINE

In [ ]:
print("=" * 70)
print("📊 RESUMEN DEL PIPELINE ETL")
print("=" * 70)

print("\n✅ FASES COMPLETADAS:")
print("  1️⃣ Extracción de datos")
print("  2️⃣ Transformación y limpieza")
print("  3️⃣ Enriquecimiento con APIs")
print("  4️⃣ Validación de calidad")
print("  5️⃣ Carga de datos procesados")

print("\n📈 MÉTRICAS:")
print(f"  📊 Registros procesados: {len(df_transformed):,}")
print(f"  📋 Columnas originales: {len(df_raw.columns)}")
print(f"  📋 Columnas finales: {len(df_transformed.columns)}")
print(f"  ➕ Columnas agregadas: {len(df_transformed.columns) - len(df_raw.columns)}")
print(f"  💹 Tasa de Churn: {(df_transformed['Churn'] == 'Yes').mean():.2%}")

print("\n📁 ARCHIVOS GENERADOS:")
print(f"  ✓ {raw_path}")
print(f"  ✓ {processed_path}")
print(f"  ✓ {parquet_path}")
print(f"  ✓ {metadata_path}")

print("\n🚀 SIGUIENTE PASO:")
print("  Ejecutar 02_EDA_Exploratory_Analysis.ipynb")
print("=" * 70)

---

## 🎯 CONCLUSIÓN

✅ Pipeline ETL ejecutado exitosamente  
✅ Datos extraídos, transformados y cargados  
✅ Calidad de datos validada  
✅ Enriquecimiento con APIs completado  

**Dataset final listo para análisis exploratorio.**

---

### 💜 Desarrollado con ❤️ por Elizabeth Díaz Familia

---